In [9]:
import seaborn as sns
import matplotlib.pyplot as plt
import os
import re
import json
import pandas as pd
from tqdm import tqdm


def load_data(results_dir: str):
    prec_at_k = pd.DataFrame(columns=["In-Layer", "Out-Layer", "Training Target", "N", "K", "Precision@K", "Method", "Context Index"])
    surprisal = pd.DataFrame(columns=["In-Layer", "Out-Layer", "Training Target", "N", "Surprisal", "Method", "Context Index"])

    for folder in tqdm(os.listdir(results_dir)):
        if ".DS_Store" in folder:
            continue
        in_layer, out_layer, training_tar, metric, method = re.findall(r"layer(\d+)to(\d+)_tk([\d])_([a-z\d\@]*)_([a-z\d]*)", folder)[0]
        result = json.load(open(os.path.join(results_dir, folder, "test_results.json"), "r"))
        context_idx = method[-1] if method.startswith("ctx") else None
        training_tar = training_tar if not method.startswith("ctx") else -1
        method = "Learned prompt" if method == "prefix" else "Fixed prompt"

        if metric == "surprisal":
            for n in range(len(result["surprisal"])):
                surprisal.loc[len(surprisal.index)] = [int(in_layer) + 1, int(out_layer), int(training_tar), int(n), float(result["surprisal"][n]), method, context_idx]
        else:
            for n in result.keys():
                for k in range(len(result[n])):
                    prec_at_k.loc[len(prec_at_k.index)] = [int(in_layer) + 1, int(out_layer), int(training_tar), int(n), k+1, float(result[n][k]), method, context_idx]
    
    return prec_at_k, surprisal

In [10]:
prec_df, surp_df = load_data("results/testing/")

100%|██████████| 449/449 [00:10<00:00, 44.60it/s]


In [11]:
linear_baseline = pd.read_csv("./results/linear-baseline-version-results.csv")
for i in linear_baseline.index:
    name = linear_baseline.loc[i, "Name"]
    in_layer, n = re.match(r"PREV_L([-?\d]+)_OUT_L27_NEXT_T(-?\d+)", name).groups()
    n = int(n) + 1
    in_layer = int(in_layer) + 1
    if in_layer == 0:
        continue
    results = float(linear_baseline.loc[i]["test/accuracy"]) / 100
    prec_df.loc[len(prec_df.index)] = [in_layer, 27, -1, n, 1, results, "Linear Vocab", None]

linear_model = pd.read_csv("./results/final-conll-linear-run.csv")
for i in linear_model.index:
    name = linear_model.loc[i, "Name"]
    in_layer, n = re.match(r"PREV_L([-?\d]+)_OUT_L27_NEXT_T(-?\d+)", name).groups()
    if in_layer == str(-1):
        continue
    n = int(n) + 1
    in_layer = int(in_layer) + 1
    results = float(linear_model.loc[i]["test/accuracy"]) / 100
    prec_df.loc[len(prec_df.index)] = [in_layer, 27, -1, n, 1, results, "Linear Model", None]

linear_surp_df = pd.read_csv("./results/linear_surprisal.csv")
surp_df = pd.concat([surp_df, linear_surp_df], ignore_index=True)

In [17]:
methods = set(prec_df["Method"])
for method in methods:
    print(method)
    sub_df = prec_df[prec_df["Method"] == method]
    sub_df = sub_df[(sub_df["Training Target"] == -1) | (sub_df["Training Target"] == 1)]
    sub_df = sub_df[sub_df["K"] == 10]
    for n in [0, 1, 2, 3]:
        print(n)
        subsub_df = sub_df[sub_df["N"] == n]
        print(subsub_df["Precision@K"].mean())
        print()
    
    print("------------------")

Linear Model
0
nan

1
nan

2
nan

3
nan

------------------
Learned prompt
0
0.8786428592034748

1
0.695214284317834

2
0.6732142844370433

3
0.7197500020265579

------------------
Fixed prompt
0
0.7967410723545721

1
0.44415178522467613

2
0.49450892955064774

3
0.5935357111905303

------------------
Linear Vocab
0
nan

1
nan

2
nan

3
nan

------------------


In [ ]:
methods = set(surp_df["Method"])
for method in methods:
    print(method)
    sub_df = surp_df[surp_df["Method"] == method]
    sub_df = sub_df[(sub_df["Training Target"] == -1) | (sub_df["Training Target"] == 1)]
    for n in [0, 1, 2, 3]:
        print(n)
        subsub_df = sub_df[sub_df["N"] == n]
        print(subsub_df["Surprisal"].mean())
        print()

    print("------------------")

In [14]:
prec_df[prec_df["Method"] == "Learned prompt"]

KeyError: 'Methods'